# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [1]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify
from sidecar import Sidecar


Define data path

In [2]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [3]:
region = 74378

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [4]:
buildings = gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet").to_crs(4326).reset_index()

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [51]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(buildings, opacity=.3)

CPU times: user 380 ms, sys: 46 ms, total: 426 ms
Wall time: 425 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [52]:
sc = Sidecar(title='buildings')

Create a Map object

In [53]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [54]:
with sc:
    display(m)

List avaialable columns

In [55]:
buildings.columns

Index(['index', 'currentUse', 'buildingNature', 'heightAboveGround', 'iid',
       'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbCCM', 'ssbCCD', 'sdbAre',
       'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo', 'stbOri', 'mtbSWR', 'libNCo',
       'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD', 'stbCeA', 'nID',
       'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [56]:
# buildings.explore()

In [76]:
column = 'stbSAl'

classifier = classify(buildings[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

In [58]:
used_keys[column]

'circular compactness of building'

In [118]:
buildings[column].describe().iloc[1:]

mean     8.943344
std      9.662884
min      0.000019
25%      1.251941
50%      5.241196
75%     13.794330
max     44.965289
Name: stbSAl, dtype: float64

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [13]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [77]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=.2)

CPU times: user 1.3 s, sys: 200 ms, total: 1.5 s
Wall time: 1.5 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [78]:
sc = Sidecar(title='tess')

Create a Map object

In [79]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [80]:
with sc:
    display(m)

List avaialable columns

In [81]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nID',
       'nodeID'],
      dtype='object')

In [102]:
column = 'stcSAl'
used_keys[column]

'street alignment of ETC'

Specify a column and pass its values into a choropleth representation within the map. 

In [103]:

classifier = classify(tess[column], 'quantiles', k=100)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [26]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [104]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(enc, opacity=.3)

CPU times: user 93.3 ms, sys: 15.3 ms, total: 109 ms
Wall time: 108 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [105]:
sc = Sidecar(title='enclosures')

Create a Map object

In [106]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [107]:
with sc:
    display(m)

List avaialable columns

In [108]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWCe', 'likWBB'],
      dtype='object')

In [125]:
column = 'likWBB'
used_keys[column]

'total of building areas within the enclosure, normalised by enclosure area'

Specify a column and pass its values into a choropleth representation within the map. 

In [126]:

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1653: UserWarning: Not enough unique values in array to form 20 classes. Setting k to 17.
  self.bins = quantile(y, k=k)


## Streets

Load data and ensure the geometries are all valid Polygons.

In [34]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [127]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 80.7 ms, sys: 13.1 ms, total: 93.8 ms
Wall time: 93.2 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [128]:
sc = Sidecar(title='streets')

Create a Map object

In [129]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [130]:
with sc:
    display(m)

List avaialable columns

In [131]:
assert np.allclose(streets['sdsLen'] , streets.geometry.length)

if 'mm_len' in streets.columns:
    assert np.allclose(streets['mm_len'] , streets.geometry.length)

In [132]:
streets.columns

Index(['geometry', '_status', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre', 'sisBpM',
       'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [133]:
streets[['sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD']].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,11876.000000,11876.000000,11876.000000,1.010500e+04,11876.000000,1.181100e+04,10146.000000,11876.000000,11876.000000,10660.000000
mean,307.629359,0.953169,338.999191,3.336419e+04,170.401987,5.487500e+05,0.058961,29.601842,0.722908,4.301254
std,558.078732,0.101226,237.645150,5.708687e+04,119.327161,4.098679e+05,0.083842,10.956701,0.210649,2.006253
min,1.758332,0.000000,6.273199,3.536984e-04,0.000000,1.311181e+03,0.000100,0.777803,0.040541,0.000000
25%,65.570235,0.956268,137.540639,5.017514e+03,93.000000,2.421871e+05,0.023237,21.360823,0.581772,2.961638
50%,119.167736,0.992331,281.132243,1.338652e+04,146.000000,4.350736e+05,0.045501,27.506520,0.735294,4.557355
75%,239.728173,0.999968,488.934188,3.592710e+04,213.000000,7.581617e+05,0.073787,37.071679,0.909091,5.752447
max,11136.453664,1.000000,3765.957781,1.050496e+06,1308.000000,2.966332e+06,3.993634,50.000000,1.000000,10.483737


Specify a column and pass its values into a choropleth representation within the map. 

In [171]:
column = 'sdsSWD'
used_keys[column]

'width deviation of street profile'

In [172]:
streets[column] = streets[column].fillna(0)

In [173]:
classifier = classify(streets[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [174]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [175]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 27.2 ms, sys: 219 μs, total: 27.4 ms
Wall time: 27.1 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [176]:
sc = Sidecar(title='nodes')

Create a Map object

In [177]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [178]:
with sc:
    display(m)

List avaialable columns

In [179]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [180]:
nodes[['mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre']].describe()

,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,nodeID,sddAre,midRea,midAre
count,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,9553.000000,8783.000000,9553.000000,9.268000e+03
mean,2.486339,0.090257,0.703328,0.092116,0.201327,0.000004,0.004018,0.006293,583.697656,0.021895,298.941753,4776.000000,38386.104882,28.919606,9.299999e+04
std,0.999710,0.050862,0.108699,0.069908,0.110069,0.000004,0.007820,0.004575,1113.959450,0.046916,448.624197,2757.857895,55242.924659,30.086879,1.062046e+05
min,1.000000,-0.333333,0.000000,0.000000,0.000000,0.000000,0.000531,0.000000,0.000000,0.000000,2.306506,0.000000,130.917440,0.000000,2.627108e+02
25%,1.000000,0.054054,0.648649,0.044444,0.133333,0.000001,0.001880,0.003096,108.539503,0.000000,82.859539,2388.000000,8410.188820,10.000000,2.548099e+04
50%,3.000000,0.086957,0.710526,0.078947,0.189189,0.000002,0.002748,0.004541,256.703812,0.000000,134.616663,4776.000000,18874.267885,22.000000,5.523687e+04
75%,3.000000,0.123077,0.769231,0.127660,0.250000,0.000005,0.004923,0.008550,564.404334,0.000000,291.378535,7164.000000,43717.353748,38.000000,1.228975e+05
max,7.000000,0.333333,0.979167,0.464286,1.000000,0.000048,0.318817,0.036858,23564.547014,0.500000,11136.453664,9552.000000,949282.266921,527.000000,1.198090e+06


In [209]:
column = 'midAre'
used_keys[column]

'total area of tess cells in node neigbhorhood'

In [210]:

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])